# PSG Complete Data Analysis - FootballDecoded

### Objetivo del Análisis

Este notebook realiza una **extracción exhaustiva y organizada** de todos los datos disponibles del **Paris Saint-Germain** para las temporadas **2023/24 y 2024/25**, utilizando el conjunto completo de wrappers desarrollados en FootballDecoded.

**Datos a extraer:**
- Estadísticas de equipo (temporada completa por competición)
- Estadísticas individuales (todos los jugadores de la plantilla)
- Eventos de partidos (goles, pases, disparos, etc.)
- Datos espaciales (coordenadas, mapas de calor, redes de pase)
- Métricas avanzadas (xG, xA, PPDA, build-up chains)
- Análisis específico del partido PSG vs Auxerre (01/09/2024)

**Fuentes de datos:**
- **FBref**: Estadísticas completas y eventos
- **Understat**: Métricas avanzadas (xGChain, PPDA, etc.)
- **WhoScored**: Datos espaciales y coordenadas

In [29]:

# ===================================================

# Dependencias base
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Union
from datetime import datetime
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Añadir path para imports
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.path.abspath('')), '..', '..'))

# FBREF WRAPPER - Prefijo 'fbref_'
from wrappers.fbref_data import (
    extract_player_season as fbref_extract_player_season,
    extract_player_match as fbref_extract_player_match,
    extract_multiple_players as fbref_extract_multiple_players,
    extract_league_players as fbref_extract_league_players,
    extract_team_season as fbref_extract_team_season,
    extract_multiple_teams as fbref_extract_multiple_teams,
    extract_match_events as fbref_extract_match_events,
    extract_league_schedule as fbref_extract_league_schedule,
    analyze_full_squad as fbref_analyze_full_squad,
    compare_teams as fbref_compare_teams,
    analyze_league as fbref_analyze_league,
    export_to_csv as fbref_export_to_csv,
    get_player as fbref_get_player,
    get_team as fbref_get_team,
    get_players as fbref_get_players,
    get_teams as fbref_get_teams,
    get_league_players as fbref_get_league_players,
    get_match_data as fbref_get_match_data,
    get_schedule as fbref_get_schedule
)

# UNDERSTAT WRAPPER - Prefijo 'understat_'
from wrappers.understat_data import (
    extract_player_season as understat_extract_player_season,
    extract_multiple_players as understat_extract_multiple_players,
    extract_team_season as understat_extract_team_season,
    extract_multiple_teams as understat_extract_multiple_teams,
    extract_shot_events as understat_extract_shot_events,
    merge_with_fbref as understat_merge_with_fbref,
    export_to_csv as understat_export_to_csv,
    get_player as understat_get_player,
    get_team as understat_get_team,
    get_squad as understat_get_squad,
    get_shots as understat_get_shots
)

# WHOSCORED WRAPPER - Prefijo 'whoscored_'
from wrappers.whoscored_data import (
    extract_match_events as whoscored_extract_match_events,
    extract_player_heatmap as whoscored_extract_player_heatmap,
    extract_defensive_events as whoscored_extract_defensive_events,
    extract_pass_network as whoscored_extract_pass_network,
    extract_shot_map as whoscored_extract_shot_map,
    extract_team_match_analysis as whoscored_extract_team_match_analysis,
    extract_match_momentum as whoscored_extract_match_momentum,
    extract_league_schedule as whoscored_extract_league_schedule,
    extract_match_missing_players as whoscored_extract_match_missing_players,
    export_to_csv as whoscored_export_to_csv,
    get_match_events as whoscored_get_match_events,
    get_player_heatmap as whoscored_get_player_heatmap,
    get_defensive_events as whoscored_get_defensive_events,
    get_pass_network as whoscored_get_pass_network,
    get_shot_map as whoscored_get_shot_map,
    get_schedule as whoscored_get_schedule,
    get_missing_players as whoscored_get_missing_players,
    get_team_analysis as whoscored_get_team_analysis,
    get_momentum as whoscored_get_momentum
)

# VISUALIZACIÓN
from visualization.pass_network import (
    create_pass_network_statsbomb,
    quick_pass_network
)

# SCRAPERS BASE (opcional)
from scrappers import FBref, Understat, WhoScored

ModuleNotFoundError: No module named 'wrappers'